## This notebook will demonstrate the huggingface API with the pretrained model, ProtBERT, to get embeddings for each of the available proteins in the dataset

In [3]:
!pip install transformers

In [1]:
%cd ..

/home/glect/pytorch/yaamformer


In [3]:
from transformers import BertForMaskedLM, BertTokenizer, pipeline
#import Auto
import pandas as pd
import numpy as np
import re
import torch
from utils import *

Seqs,Labels = get_dataset('./data/YAAM.txt')


### Loading huggingface API for the ProtBERT model

In [ ]:
from transformers import BertForMaskedLM, BertTokenizer, pipeline
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
model = BertForMaskedLM.from_pretrained("Rostlab/prot_bert")
model.to('cuda')

Load the pre-trained model, pre-process the data, and collect the outputs from the model. Take the LAST hidden layer to be used for token classification. This is the protocol for using the feature embeddings for token classification

In [ ]:
from tqdm import tqdm
representations = []
for i in tqdm(range(len(yaam))):
    try:
    #print(i)
    sequence_Example = yaam['seqs'][i]
    #print(sequence_Example)
    sequence_Example = re.sub(r"[UZOB]", "X", sequence_Example)
    sequence_Example = ' '.join(sequence_Example.replace('\n',''))
    encoded_input = tokenizer(sequence_Example, return_tensors='pt')
    with torch.no_grad():
        output = model(**encoded_input.to('cuda'),output_hidden_states=True)

    representations.append(output['hidden_states'][-1])  # get last layers hidden state for classificaiton

The tokens can be used now for standard classification with ML tools like random forests